安裝及載入所需資料

In [ ]:
!pip install pysam
import pysam
import math
import gdown
import numpy as np
import pickle
!gdown --folder 1vVfuNh9coeT3U3qSAJBlyXA7WZspDPSN

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.9/21.9 MB 42.1 MB/s eta 0:00:00
Retrieving folder contents
Retrieving folder 1ABzlrPT7-fsz7ccIJRQML7_OdkMXut63 __MACOSX
Processing file 1743PVD5_s-qCSQ0QoKQ96IaoFi6RJJ_N ._D2-ChIP
Retrieving folder 1T5T9R8Nw1p4-ZkzPpfa2RrmD0n-qgQcc D2-ChIP
Processing file 1Zh0B3OglcYjDUPdAZheJW3JXaxp4wV_m ._.DS_Store
Retrieving folder 1feWmYk0pOKRmHX3iN_BDC6gh7mJFuNVU D2-ChIP
Processing file 1g6MvpnSeND8qKR_Bhj3jTCVB5UHcrGzn .DS_Store
Retrieving folder 1FhiNJhlS91IGhykvIa-QthwLbMf6F9cd D2
Processing file 128sJrNUAkFjgeUdIlNobhXyOHEYpFpCT 4.fa
Processing file 1YD3qUQCemCpF-as-D3FwFstFk8PnEEHp ans_chr4.pickle
Processing file 1XlD69g_gWimPmcv4CYIbGi_w1jOpCIwA SRR060778.fastq.bam
Processing file 1unN05Oyu7yrmDI2VI-dUXXXJqKr7jhAN test_ChIP.py
Processing file 1RUCQEeg2eYjTxbZNnCUF-asn_3TXAxlk X.fa
Retrieving folder contents completed
Building directory structure
Building directory structure completed
Downloading...
From: https://drive.google.com/uc?id=1743PVD5

讀取X.fa，將其變為ref_seq

In [ ]:
def read_fasta_file(file_path):
    sequences = ""
    with open(file_path, 'r') as f:
        for line in f:
            sequences += line.strip()
    return sequences

fasta_file_path = '/content/D2-ChIP/D2-ChIP/D2/4.fa'

ref_seq = read_fasta_file(fasta_file_path)
ref_seq = ref_seq.split('Dmel;')[1] #保留ATCG部分
map_seq = np.zeros(len(ref_seq))+1e-6 #創建一個長度和ref_seq相同的矩陣存取mapping_count

讀取.bam檔，透過迴圈篩選X染色體，計算read_count和更新map_seq

In [ ]:
doc_path = '/content/D2-ChIP/D2-ChIP/D2/SRR060778.fastq.bam'
sam_file = pysam.AlignmentFile(doc_path,'rb')
read_cnt = 1e-6

for read in sam_file:

  start=read.pos # zero-base start (pass read if line.pos ==-1)
  seq=read.query_sequence # (get seq)
  chromosome=read.reference_name # (get chromosome)

  if chromosome=='4':
    read_cnt+=1
    end = start+len(seq)-1
    map_seq[start:end+1] = map_seq[start:end+1] + 1

SRR060778.26	0	#4	659561	255	36M	*	0	0	AAACAGAAATTAGAAATTAATATCTATTGCCAGAAT	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40])	[('XA', 0), ('MD', '36'), ('NM', 0)]
SRR060778.38	16	#4	535788	255	36M	*	0	0	TTGTATGAATGTTATCAATCCGAAGCAAATACAAGT	array('B', [40, 10, 40, 36, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 6, 40, 40, 40, 40, 40, 3, 40, 40, 40, 40, 40])	[('XA', 2), ('MD', '24T5C5'), ('NM', 2)]
SRR060778.190	16	#4	497919	255	36M	*	0	0	TCAGTTTGTATGTTGACAAGGGAATGTGTACCCAAC	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40])	[('XA', 0), ('MD', '36'), ('NM', 0)]
SRR060778.196	0	#4	1330589	255	36M	*	0	0	GCTGGGCTTGGGTTTGTGTCTTGGGCGGCATGCAAT	array('B', [40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40, 40,

用 map_seq 和 read_cnt 得到 ChIP score

In [ ]:
chip_seq = (np.log((map_seq/read_cnt)))
chip_seq = list(chip_seq) # array 轉 list

將ChIP score儲存成新的pickle檔案

In [ ]:
Ans_array = chip_seq # data type=list !!
print(Ans_array[:100])

with open('ans.pickle','wb') as f:
  pickle.dump(Ans_array,f)

[-24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231, -24.67208353822231,

驗證pickle檔是否正確 (請輸入答案路徑)

In [ ]:
arr=[]
arr2=[]
p=input('ans.pickle path=')
with open(p,'rb') as f:
    arr=pickle.load(f)
with open('/content/ans.pickle','rb') as f:
    arr2=pickle.load(f)

arr=np.array(arr, dtype = 'uint8')
arr2=np.array(arr2, dtype = 'uint8')

print((arr == arr2).all())